In [ ]:
from run_aclsum import *

Try a range of different questions. Besides extracting information into a taxonomy, I am converting the following datasets into test questions:
+ ACLSum (Summarize the Challenge, Approach, and Outcome in the paper)
+ SciREX (extract the main results of a scientific article including Dataset, Metric, Task and Method)
+ arxivDIGESTables (given a table schema for literature survey and extract targeted values from scientific papers to fill in the table)

In [ ]:
with open('words_alpha.txt') as f:
    words_alpha = set(f.read().splitlines())
doc_manager = DocManager(word_vocab=words_alpha)

# ACLSum

## Normalize Data Samples

In [ ]:
from aclsum import ACLSum

# Load per split ("train", "val", "test")
split = "train"
train = ACLSum(split)

aclsum_dataset = list[dict]()
for doc in train:
    aclsum_dataset.append(Sample(
        doc_file=f'https://aclanthology.org/{doc.id}.pdf',
        doc_strs=[
            'Abstract', 
            DocManager.remove_citations(DocManager.remove_space_before_punct(' '.join(doc.get_all_sentences(['abstract'])))), 
            'Introduction', 
            DocManager.remove_citations(DocManager.remove_space_before_punct(' '.join(doc.get_all_sentences(['introduction'])))), 
            'Conclusion', 
            DocManager.remove_citations(DocManager.remove_space_before_punct(' '.join(doc.get_all_sentences(['conclusion'])))), 
        ],
        outline='Abstract\nIntroduction\nConclusion',
        question_types=['challenge', 'approach', 'outcome'],
        questions={
            'challenge': 'Summarize the challenge of the paper, which is the current situation faced by the researcher. It will normally include a Problem Statement, the Motivation, a Hypothesis and/or a Goal.', 
            'approach': 'Summarize the approach of the paper: How they intend to carry out the investigation, comments on a theoretical model or framework.', 
            'outcome': 'Summarize the outcome of the paper: Overall conclusion that should reject or support the research hypothesis.'
        },
        answers={
            'challenge': doc.summaries['challenge'], 
            'approach': doc.summaries['approach'], 
            'outcome': doc.summaries['outcome']
        },
        extractions={
            'challenge': [DocManager.remove_citations(DocManager.remove_space_before_punct(sent)) for sent in doc.get_all_highlighted_sentences('challenge')],
            'approach': [DocManager.remove_citations(DocManager.remove_space_before_punct(sent)) for sent in doc.get_all_highlighted_sentences('approach')],
            'outcome': [DocManager.remove_citations(DocManager.remove_space_before_punct(sent)) for sent in doc.get_all_highlighted_sentences('outcome')],
        }
    ).model_dump())
    
with jsonlines.open(f'{ACLSUM_DIR}/{split}_dataset.jsonl', 'w') as f_out:
    f_out.write_all(aclsum_dataset)

## Experiments

In [ ]:

# Dataset config
split = 'train'

load_from_pdf = False
# load_from_pdf = True

# Retrieval config
# retrieval_method = 'rag'
retrieval_method = 'rag_base'
# retrieval_method = 'gen'

# Chunk config
# sent_chunk = True
# max_seq_len = None
# k = 10
# sent_chunk = False
# max_seq_len = None
# k = 3
sent_chunk = False
max_seq_len = 100
k = 10

with jsonlines.open(f'{ACLSUM_DIR}/{split}_dataset.jsonl') as f_in:
    aclsum_dataset = [Sample.model_validate(line) for line in f_in]

In [ ]:
missed_samples = [sid for sid, sample in enumerate(aclsum_dataset) if not os.path.exists(f"{ACLSUM_PDF_DIR}/outline_{sample.doc_file.split('/')[-1].replace('.pdf', '.txt')}")]
missed_samples

#### Evaluate Experiments

In [ ]:
print('gen', f'split--{split}', f'load_from_pdf--{load_from_pdf}', f'sent_chunk--{sent_chunk}', f'max_seq_len--{max_seq_len}')
for question_type in ['challenge', 'approach', 'outcome']:
    with open(f'{ACLSUM_EVALUATION_DIR}/{retrieval_method}/{split}_{question_type}_{load_from_pdf}_{sent_chunk}_{max_seq_len}.json') as f_in:
        eval_results = json.load(f_in)
        print('question_type', question_type)
        print('recall', np.mean([result['recall'] for result in eval_results[:]]))
        print('precision', np.mean([result['precision'] for result in eval_results[:]]))
        print('f1', np.mean([result['f1'] for result in eval_results[:]]))
        print('')

In [ ]:
print('rag', f'split--{split}', f'load_from_pdf--{load_from_pdf}', f'sent_chunk--{sent_chunk}', f'max_seq_len--{max_seq_len}', f'k--{k}')
for question_type in ['challenge', 'approach', 'outcome']:
    eval_file = get_eval_file(retrieval_method, split, question_type, load_from_pdf, sent_chunk, max_seq_len, k, is_temp=False)
    with open(eval_file) as f_in:
        eval_results = json.load(f_in)
        print('question_type', question_type)
        print('recall', np.mean([result['recall'] for result in eval_results[:]]))
        print('precision', np.mean([result['precision'] for result in eval_results[:]]))
        print('f1', np.mean([result['f1'] for result in eval_results[:]]))
        print('')

## Observation

In [ ]:
retrieval2configs = {
    'rag': [
        {
            'load_from_pdf': False, 
            'sent_chunk': True, 
            'max_seq_len': None, 
            'k': 10
        },
        {
            'load_from_pdf': False, 
            'sent_chunk': False, 
            'max_seq_len': 100, 
            'k': 10
        }
    ],
    'gen': [
        {
            'load_from_pdf': False, 
            'sent_chunk': True, 
            'max_seq_len': None, 
            'k': None
        },
        {
            'load_from_pdf': True, 
            'sent_chunk': True, 
            'max_seq_len': None, 
            'k': None
        }
    ]
}
retrieval2configs['rag_base'] = retrieval2configs['rag']


sid = 4
split = 'train'
question_type = 'challenge'
with jsonlines.open(f'{ACLSUM_DIR}/{split}_dataset.jsonl') as f_in:
    aclsum_dataset = [Sample.model_validate(line) for line in f_in]

sample = aclsum_dataset[sid]
test2sents = dict[str, list[str]]()
test2process = dict[str, list[dict]]()
test2chunks = dict[str, list[str]]()

for retrieval_method, retrieval_configs in retrieval2configs.items():
    for retrieval_config in retrieval_configs:
        load_doc_manager(doc_manager, sample, retrieval_config['load_from_pdf'])
        doc_manager.build_chunks(sent_chunk=retrieval_config['sent_chunk'], max_seq_length=retrieval_config['max_seq_len'])
        
        unique_ngram2sent = get_sent_index([sent.text for section in doc_manager.sections if section.section_nlp_local for sent in section.section_nlp_local.sents])
        if retrieval_config['load_from_pdf']:
            valid_sent_ids = get_sent_ids([sent for block in sample.doc_strs if block not in ['Abstract', 'Introduction', 'Conclusion'] for sent in spacy_sent_tokenize(doc_manager.nlp, block)], unique_ngram2sent)
            if -1 in valid_sent_ids:
                print(f'Invalid sent id in sample {sid}, retrieval_config {retrieval_config}, {valid_sent_ids.count(-1)}/{len(valid_sent_ids)}')
                valid_sent_ids = [sent_id for sent_id in valid_sent_ids if sent_id > -1]
            valid_sent_ids = set(valid_sent_ids)
        else:
            valid_sent_ids = set(range(max(sent_id for ngram, (sent_id, sent) in unique_ngram2sent.items()) + 1))
        
        process, retrieved_sents = get_sents_and_process(
            doc_manager=doc_manager,
            retrieval_method=retrieval_method,
            split=split,
            sid=sid,
            question_type=question_type,
            **retrieval_config
        )
        valid_retrieved_sents = [sent for sent_id, sent in zip(get_sent_ids(retrieved_sents, unique_ngram2sent), retrieved_sents) if sent_id in valid_sent_ids]
        test_name = f"{retrieval_method}_{retrieval_config['load_from_pdf']}_{retrieval_config['sent_chunk']}_{retrieval_config['max_seq_len']}"
        test2sents[test_name] = valid_retrieved_sents
        test2process[test_name] = process
        test2chunks[test_name] = [chunk.page_content for chunk in doc_manager.chunks]
        
test2label = {
    'rag_False_True_None': 'rag_sent', 
    'rag_False_False_100': 'rag_100', 
    'gen_False_True_None': 'gen', 
    'gen_True_True_None': 'gen_full', 
    'rag_base_False_True_None': 'rag_sent_base', 
    'rag_base_False_False_100': 'rag_100_base',
    'GOLD': 'GOLD'
}

label2order = {
    'rag_sent': 6, 
    'rag_100': 5, 
    'gen': 4, 
    'gen_full': 3, 
    'rag_sent_base': 2, 
    'rag_100_base': 1, 
    'GOLD': 0, 
}

load_doc_manager(doc_manager, sample, False)
source_sents = [sent.text for section in doc_manager.sections if section.section_nlp_local for sent in section.section_nlp_local.sents]
unique_ngram2sent = get_sent_index(source_sents)

sent_id2labels = [[] for _ in range(max(sent_id for ngram, (sent_id, sent) in unique_ngram2sent.items()) + 1)]
for test_name, test_label in test2label.items():
    if test_name == 'GOLD':
        test_sents = sample.extractions[question_type]
    else:
        test_sents = test2sents[test_name]
    for sent_id in get_sent_ids(test_sents, unique_ngram2sent):
        sent_id2labels[sent_id].append(test_label)
        

In [ ]:
# skipped_labels = {'gen_full', 'rag_sent', 'rag_100'}
# skipped_labels = {'gen_full', 'gen'}
skipped_labels = {'gen_full', 'gen', 'rag_sent', 'rag_100', 'rag_sent_base', 'rag_100_base'}
with open(f'observations_{sid}.txt', 'w') as f_out:
    for sent_id, sent in enumerate(source_sents):
        f_out.write(f'{sent} --- {", ".join(sorted([label for label in sent_id2labels[sent_id] if label not in skipped_labels], key=lambda x: label2order[x]))}\n')

In [ ]:
eval_file = get_eval_file('rag', split, question_type, load_from_pdf=False, sent_chunk=False, max_seq_len=100, k=10, is_temp=False)
with open(eval_file) as f_in:
    eval_results = json.load(f_in)
eval_results[sid]

In [ ]:
for eval_result in eval_results:
    if eval_result['recall']:
        print(eval_result)

In [ ]:
test2chunks.keys()

In [ ]:
test2chunks['rag_base_False_False_100']

In [ ]:
sample.extractions['challenge']

In [ ]:
sample.questions['challenge']

In [ ]:
doc_manager.build_chunks(sent_chunk=True, max_seq_length=None)

In [ ]:
doc_manager.vectorstore.similarity_search('Limitations of the current methods')

In [ ]:
doc_manager.vectorstore.similarity_search('We propose a new method to solve this problem.')

In [ ]:
content = PARAGRAPH_SEP.join([f'Chunk {chunk.metadata["chunk_id"]}: {chunk.page_content}' for chunk in doc_manager.chunks])
similar_chunk_prompt = f'Below are text chunks from a paper:\n\n\n\n{content}\n\n\n\nSelect the Chunk ids that express similar general meaning as the following statement: \n\n{"Previous [Method] has been used for [Task]."}\n\nReturn only the selected chunk ids separated by commas, e.g. "1, 3, 5".'
chat_completion = doc_manager.client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": similar_chunk_prompt,
        }
    ],
    model=doc_manager.tool_llm,
)
content = chat_completion.choices[0].message.content
content

In [ ]:
doc_manager.chunks[21]

In [ ]:
import random
split = 'train'
question_type = 'challenge'
with jsonlines.open(f'{ACLSUM_DIR}/{split}_dataset.jsonl') as f_in:
    aclsum_dataset = [Sample.model_validate(line) for line in f_in]
eval_metrics = EvalMetrics()
eval_results = list[dict]()
for sid, sample in enumerate(tqdm(aclsum_dataset)):
    load_doc_manager(doc_manager, sample, False)
    doc_manager.build_chunks(sent_chunk=True, max_seq_length=None)
    unique_ngram2sent = get_sent_index([sent.text for section in doc_manager.sections if section.section_nlp_local for sent in section.section_nlp_local.sents])
    chunks = [chunk.page_content for chunk in doc_manager.chunks]
    # random.shuffle(chunks)
    content = PARAGRAPH_SEP.join([f'Chunk {chunk_id}: {chunk}' for chunk_id, chunk in enumerate(chunks)])
    selected_chunk_ids = set[int]()
    question2chunk_ids = dict[str, list[int]]()
    for question in [
        # "[Task] is widely studied in the research.",
        "Introduction of [Task].",
        # "Previous [Method] has been used for [Task].",
        # "Previous [Method] has drawbacks.",
        "[Method] has been used for [Task].",
        "[Method] has limitations.",
        # "We propose a new [Method] to solve this problem.",
    ]:
        similar_chunk_prompt = f'Below are text chunks from a paper:\n\n\n\n{content}\n\n\n\nRank the **TOP 5** Chunk ids that belong to the following topic: \n\n{question}\n\nReturn only the selected chunk ids separated by commas, e.g. "1, 3, 5".'
        chat_completion = doc_manager.client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": similar_chunk_prompt,
                }
            ],
            model=doc_manager.tool_llm,
        )
        try:
            selected_chunk_ids.update(map(int, chat_completion.choices[0].message.content.split(', ')))
            question2chunk_ids[question] = list(map(int, chat_completion.choices[0].message.content.split(', ')))
        except:
            question2chunk_ids[question] = []
    
    retrieved_sents = [sent for chunk_id in selected_chunk_ids if chunk_id < len(chunks)  for sent in spacy_sent_tokenize(doc_manager.nlp, chunks[chunk_id])]
    if not retrieved_sents:
        eval_result = {'f1': 0, 'precision': 0, 'recall': 0}
    else:
        retrieved_sent_ids = get_binary_sent_ids(retrieved_sents, unique_ngram2sent)
        gold_sent_ids = get_binary_sent_ids(sample.extractions[question_type], unique_ngram2sent)
        
        eval_result:dict[str, Any] = eval_metrics.eval_precision_recall_f1(predictions=retrieved_sent_ids, references=gold_sent_ids)
    eval_result.update({'sid': sid, 'sent_ids': retrieved_sents, 'question2chunk_ids': question2chunk_ids})
    eval_results.append(eval_result)

In [ ]:
with open('temp_eval.json', 'w') as f_out:
    json.dump(eval_results, f_out)

In [ ]:
with open('temp_eval.json') as f_in:
    eval_results = json.load(f_in)

In [ ]:
print('recall', np.mean([result['recall'] for result in eval_results[:]]))
print('precision', np.mean([result['precision'] for result in eval_results[:]]))
print('f1', np.mean([result['f1'] for result in eval_results[:]]))

In [ ]:
for eval_result in eval_results:
    if eval_result['recall'] < 0.5:
        print(eval_result['sid'], eval_result['recall'], eval_result['precision'], eval_result['f1'])

In [ ]:
aclsum_dataset[1].extractions['challenge']

In [ ]:
for eval_result, sample in zip(eval_results, aclsum_dataset):
    eval_result['missing'] = set(sample.extractions['challenge']).difference(eval_result['sent_ids'])
    eval_result['shared'] = set(sample.extractions['challenge']).intersection(eval_result['sent_ids'])
    eval_result['extra'] = set(eval_result['sent_ids']).difference(sample.extractions['challenge'])

In [ ]:
load_doc_manager(doc_manager, aclsum_dataset[0], False)
doc_manager.build_chunks(sent_chunk=True, max_seq_length=None)

In [ ]:
eval_results[0]['missing']

In [ ]:
eval_results[0]['extra']

In [ ]:
eval_results[0]['shared']

In [ ]:
chunks = [chunk.page_content for chunk in doc_manager.chunks]
[sent for chunk_id in eval_results[0]['question2chunk_ids']['[Method] has limitations.'] if chunk_id < len(chunks)  for sent in spacy_sent_tokenize(doc_manager.nlp, chunks[chunk_id])]

In [ ]:
aclsum_dataset[0].answers['challenge']

In [ ]:
aclsum_dataset[4].questions['outcome']

In [ ]:
# context = '''Results show that the proposed system outperforms significantly other stateof-the-art QE systems. This study is part of a bigger, ongoing project, aiming to develop a real-time QE system for Web search, where simplicity is the key to the success. Thus, what we learned from this study is particularly encouraging.'''

# context = '''These models are trained on pairs of user queries and the titles of clicked documents using EM. Second, we present a ranker-based QE system, the heart of which is a MRF-based ranker in which the lexicon models are incorporated as features. We perform experiments on the Web search task using a real world data set.'''

# context = '''The experimental results show that when implementing the sequence modeling layer with a single-layer Bi-LSTM, our method achieves considerable improvements over the state-of-theart methods in both inference speed and sequence labeling performance.'''

context = '''Experimental studies on four benchmark Chinese NER datasets show that our method achieves an inference speed up to 6.15 times faster than those of state-ofthe-art methods, along with a better performance. The experimental results also show that the proposed method can be easily incorporated with pre-trained models like BERT.'''

question = aclsum_dataset[4].questions['outcome']

grade_doc_prompt = f"""You are a grader assessing relevance of a retrieved document to a user question.

Here is the retrieved document:

{context}

Here is the user question:

{question}

If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant.
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. Briefly explain your reasoning for the grade."""

chat_completion = doc_manager.client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": grade_doc_prompt,
        }
    ],
    model=doc_manager.tool_llm,
)

chat_completion.choices[0].message.content

In [ ]:
context = '''In this work, we propose a simple but effective method for incorporating the word lexicon into the character representations. This method avoids designing a complicated sequence modeling architecture, and for any neural NER model, it requires only subtle adjustment of the character representation layer to introduce the lexicon information. Experimental studies on four benchmark Chinese NER datasets show that our method achieves an inference speed up to 6.15 times faster than those of state-ofthe-art methods, along with a better performance. The experimental results also show that the proposed method can be easily incorporated with pre-trained models like BERT.'''

question = aclsum_dataset[4].questions['outcome']

grade_doc_prompt = f"""You are a grader assessing relevance of a retrieved document to a user question.

Here is the retrieved document:

{context}

Here is the user question:

{question}

If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant.
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. Briefly explain your reasoning for the grade."""

chat_completion = doc_manager.client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": grade_doc_prompt,
        }
    ],
    model=doc_manager.tool_llm,
)

chat_completion.choices[0].message.content

## SciREX

+ Salient Entity Extraction
  + Extract the salient Dataset, Method, Task and Metric of the paper.
  + An entity is extracted if one of its mentions is being returned.
+ Salient Entity Mention Extraction
  + Extract the sentences where a salient entity's mention appear.
  + An entity mention is extracted if the sentence containing the mention is extracted.
+ Salient N-ary Relation Extraction
  + Extract the Dataset, Method, Task and Metric tuples that are bounded together.

### Dataset Observation and Preparation

In [ ]:
with jsonlines.open('../../data/SciREX/train.jsonl') as f_in:
    scirex_dataset = list(f_in)

In [ ]:
test_sample = scirex_dataset[0]

In [ ]:
test_sample.keys()

In [ ]:
test_sample['n_ary_relations']

In [ ]:
test_sample['doc_id']

In [ ]:
test_sample['words'][12]

In [ ]:
import requests
from time import sleep
from tqdm import tqdm


        

failed_ids = []
for sample in tqdm(scirex_dataset[2:]):
    paper_meta = requests.get(f"https://api.semanticscholar.org/graph/v1/paper/{sample['doc_id']}", params={'fields': 'externalIds'}).json()
    while 'externalIds' not in paper_meta:
        sleep(10)
        paper_meta = requests.get(f"https://api.semanticscholar.org/graph/v1/paper/{sample['doc_id']}", params={'fields': 'externalIds'}).json()
    if 'ArXiv' not in paper_meta['externalIds']:
        failed_ids.append(sample['doc_id'])
        continue
    download_file(f"https://arxiv.org/pdf/{paper_meta['externalIds']['ArXiv']}", f"../../data/SciREX/pdfs/{sample['doc_id']}.pdf")
    sleep(2)
with open('../../data/SciREX/failed_ids.txt', 'w') as f_out:
    f_out.write('\n'.join(failed_ids))

In [ ]:
doc_manager = DocManager()

In [ ]:
test_file = f"../../data/SciREX/pdfs/{test_sample['doc_id']}.pdf"
if os.path.exists(test_file):
    doc_manager.load_doc(test_file)

In [ ]:
print(doc_manager.outline)

In [ ]:
doc_manager.get_section_by_header('6. Conclusion').text

In [ ]:
test_file

In [ ]:
doc_manager.sections[5].text

In [ ]:
doc_manager.sections[3].blocks[0]

In [ ]:
# doc_strs = list[str]()
# for section_range in test_sample['sections']:
#     section_words = test_sample['words'][section_range[0]:section_range[1]]
#     if section_words[0] == 'section' and section_words[1] == ':':
#         section_words = section_words[2:]
#     doc_strs.append(' '.join(section_words))
'https://arxiv.org/pdf/2210.14427'
doc_manager.load_doc(doc_file='https://arxiv.org/pdf/1611.08323')

In [ ]:
test_sample['sentences']

In [ ]:
test_sample['method_subrelations']

## Self-RAG

## Plan and Solve

## CoT